In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_data = pd.read_csv("/content/drive/MyDrive/indian_food.csv")

In [ ]:
data.loc[data['name'].isin(['Copra paak','Puttu','Kansar']),'flavor_profile']='sweet'
data.loc[data['flavor_profile'] == '-1', 'flavor_profile'] = "other"

In [ ]:
data.tail(10)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region,sugar,...,Tripura,Uttar Pradesh,Uttarakhand,West Bengal,Central,East,North,North East,South,West
245,pani pitha,"tea leaves, white sesame seeds, dry coconut, s...",vegetarian,10,20,other,main course,Assam,North East,0,...,False,False,False,False,False,False,False,True,False,False
246,payokh,"basmati rice, rose water, sugar, clarified but...",vegetarian,-1,-1,sweet,dessert,Assam,North East,1,...,False,False,False,False,False,False,False,True,False,False
247,prawn malai curry,"coconut milk, prawns, garlic, turmeric, sugar",non-vegetarian,15,50,spicy,main course,West Bengal,East,1,...,False,False,False,True,False,True,False,False,False,False
248,red rice,"red pepper, red onion, butter, watercress, oli...",vegetarian,-1,-1,other,main course,All State,All Region,0,...,False,False,False,False,False,False,False,False,False,False
249,shukto,"green beans, bitter gourd, ridge gourd, banana...",vegetarian,10,20,spicy,main course,West Bengal,East,0,...,False,False,False,True,False,True,False,False,False,False
250,til pitha,"glutinous rice, black sesame seeds, gur",vegetarian,5,30,sweet,dessert,Assam,North East,0,...,False,False,False,False,False,False,False,True,False,False
251,bebinca,"coconut milk, egg yolks, clarified butter, all...",vegetarian,20,60,sweet,dessert,Goa,West,0,...,False,False,False,False,False,False,False,False,False,True
252,shufta,"cottage cheese, dry dates, dried rose petals, ...",vegetarian,-1,-1,sweet,dessert,Jammu & Kashmir,North,0,...,False,False,False,False,False,False,True,False,False,False
253,mawa bati,"milk powder, dry fruits, arrowroot powder, all...",vegetarian,20,45,sweet,dessert,Madhya Pradesh,Central,0,...,False,False,False,False,True,False,False,False,False,False
254,pinaca,"brown rice, fennel seeds, grated coconut, blac...",vegetarian,-1,-1,sweet,dessert,Goa,West,0,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# Performing One hot encoding
discrete_df = pd.get_dummies(data,columns=['ingredients','diet','flavor_profile'])
discrete_df

,name,prep_time,cook_time,course,state,region,sugar,ginger,garam masala,urad dal,...,"ingredients_yogurt, fresh coconut, sesame seeds, semolina, gram flour","ingredients_yogurt, ginger, curry leaves, baking soda, green chilli","ingredients_yogurt, milk, nuts, sugar",diet_non-vegetarian,diet_vegetarian,flavor_profile_bitter,flavor_profile_other,flavor_profile_sour,flavor_profile_spicy,flavor_profile_sweet
0,balu shahi,45,25,dessert,West Bengal,East,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
1,boondi,80,30,dessert,Rajasthan,West,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
2,gajar ka halwa,15,60,dessert,Punjab,North,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
3,ghevar,15,30,dessert,Rajasthan,West,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
4,gulab jamun,15,40,dessert,West Bengal,East,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,til pitha,5,30,dessert,Assam,North East,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
251,bebinca,20,60,dessert,Goa,West,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
252,shufta,-1,-1,dessert,Jammu & Kashmir,North,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
253,mawa bati,20,45,dessert,Madhya Pradesh,Central,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True


In [ ]:
target= data['course']
df= discrete_df.drop(columns=['course','state','region','name'],axis=1)
df

,prep_time,cook_time,sugar,ginger,garam masala,urad dal,ghee,curry leaves,jaggery,rice flour,...,"ingredients_yogurt, fresh coconut, sesame seeds, semolina, gram flour","ingredients_yogurt, ginger, curry leaves, baking soda, green chilli","ingredients_yogurt, milk, nuts, sugar",diet_non-vegetarian,diet_vegetarian,flavor_profile_bitter,flavor_profile_other,flavor_profile_sour,flavor_profile_spicy,flavor_profile_sweet
0,45,25,1,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
1,80,30,1,0,0,0,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
2,15,60,1,0,0,0,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
3,15,30,1,0,0,0,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
4,15,40,1,0,0,0,1,0,0,0,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,5,30,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
251,20,60,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
252,-1,-1,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True
253,20,45,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,True


In [ ]:
# Converting target string to numeric labels

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(data['course'])
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 3, 2, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2,
       1, 1, 2, 2, 1, 2, 0, 1, 0, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 0,
       2, 2, 1, 1, 2, 1, 1, 2, 0, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
# Spliting data inro training set and testing set

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(df,label, test_size=0.2)

In [ ]:
# implementing classification algorithm
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

In [ ]:
# Checking accuracy
accuracy = model.score(X_test, Y_test)
print(accuracy)
from sklearn.metrics import classification_report
print(classification_report(Y_test, ypred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         1

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

clf = RandomForestClassifier(n_estimators=2 ,min_samples_split=3 , min_samples_leaf=2)
clf.fit(X_train,Y_train)
pred_clf = clf.predict(X_test)
# sklearn.metrics.accuracy_score(Y_test,pred_clf)

In [ ]:
# pred_clf

accuracy = clf.score(X_test, Y_test)
print(accuracy)
print(classification_report(Y_test, pred_clf))

0.9411764705882353
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.96      0.93      0.95        29
           2       1.00      1.00      1.00         7
           3       0.00      0.00      0.00         1

    accuracy                           0.94        51
   macro avg       0.71      0.73      0.72        51
weighted avg       0.93      0.94      0.93        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {"n_estimators":[2,5,10,20],"min_samples_split":[2,3],"min_samples_leaf":[1,2,3]}
grid_search = GridSearchCV(estimator=clf,param_grid=param_grid)
grid_search.fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(estimator=RandomForestClassifier(min_samples_leaf=2,
                                              min_samples_split=3,
                                              n_estimators=2),
             param_grid={'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3],
                         'n_estimators': [2, 5, 10, 20]})

In [ ]:
grid_search.best_params_

{'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 10}

In [ ]:
clf = RandomForestClassifier(n_estimators=20 ,min_samples_split=2 , min_samples_leaf=1)
clf.fit(X_train,Y_train)
pred_clf = clf.predict(X_test)
accuracy = clf.score(X_test, Y_test)
print(accuracy)
print(classification_report(Y_test, pred_clf))

0.9411764705882353
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.96      0.93      0.95        29
           2       1.00      1.00      1.00         7
           3       0.00      0.00      0.00         1

    accuracy                           0.94        51
   macro avg       0.71      0.73      0.72        51
weighted avg       0.93      0.94      0.93        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, X_test, Y_train, Y_test are your train and test data
# Initialize the KNN classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # You can specify the number of neighbors here

# Train the KNN classifier
knn_clf.fit(X_train, Y_train)

# Make predictions on the test data
pred_knn = knn_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, pred_knn)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(Y_test, pred_knn))


Accuracy: 0.6862745098039216
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.64      0.64        14
           1       0.71      0.86      0.78        29
           2       0.50      0.14      0.22         7
           3       0.00      0.00      0.00         1

    accuracy                           0.69        51
   macro avg       0.46      0.41      0.41        51
weighted avg       0.65      0.69      0.65        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import LinearRegression

# Assuming X_train and Y_train are your training features and target variable respectively
# Assuming X_test is your test features

# Initialize the Linear Regression model
linear_reg = LinearRegression()

# Train the model on the training data
linear_reg.fit(X_train, Y_train)

# Make predictions on the test data
pred_linear_reg = linear_reg.predict(X_test)


In [ ]:
accuracy = linear_reg.score(X_test, Y_test)
print(accuracy)
print(classification_report(Y_test, pred_clf))


0.8413862994452332
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.96      0.93      0.95        29
           2       1.00      1.00      1.00         7
           3       0.00      0.00      0.00         1

    accuracy                           0.94        51
   macro avg       0.71      0.73      0.72        51
weighted avg       0.93      0.94      0.93        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
